## xRAG Tutorial

Retrieval-augmented Geneneration (RAG) aims to combine a parametric Large Language Model (LLM) with a non-parametric datastore, where long-tailed, domain-specific and up-to-date knowledge could be retrieved and "perceived" by LLM. RAG substantially extend the boundary of LLM, while at the cost of additional latency:
- similarity search over a potentially large datastore
- extended context for LLM to process

Today's focus is the latter and we propose a framework called xRAG which compresses the context length of document to only 1 token while perserving strong performance. Below is a comparison between traditional RAG and our proposed xRAG.

<img src="assets/framework.jpg" alt="xRAG">

## LLM without retrieval augmentation
Let's get started! Suppose we have such a question for LLM: `What company advertised itself with the slogan "We'll leave a light on for you"?` (The right answer is **Motel 6**, as shown in this [wiki page](https://en.wikipedia.org/wiki/Motel_6))


Although LLM is very powerful (better than me), it couldn't recall every factual knowledge with 100% accuracy, so it would hallucinate. Let's verify step by step:

First, we need to import necessary packages.

In [1]:
## third-party
from transformers import AutoTokenizer
import torch

## own
from src.model import SFR,XMistralForCausalLM
from src.language_modeling.utils import get_retrieval_embeds,XRAG_TOKEN

/home/azureuser/miniconda3/lib/python3.9/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


Download the LLM. In this case, we download from `Hannibal046/xrag-7b`, this is a `mistralai/Mistral-7B-Instruct-v0.2` model with an extra modality bridge that 
project the retrieval feature into the LLM representation space.

In [2]:
device = torch.device("cuda:1")
llm_name_or_path = "Hannibal046/xrag-7b"
llm = XMistralForCausalLM.from_pretrained(llm_name_or_path,torch_dtype = torch.bfloat16,low_cpu_mem_usage = True,).to(device).eval()
llm_tokenizer = AutoTokenizer.from_pretrained(llm_name_or_path,add_eos_token=False,use_fast=False,padding_side='left')

## here, XRAG_TOKEN is just a place holder
llm.set_xrag_token_id(llm_tokenizer.convert_tokens_to_ids(XRAG_TOKEN))
print(XRAG_TOKEN)

/home/azureuser/miniconda3/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/home/azureuser/miniconda3/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


<xRAG>


Let's see how `mistralai/Mistral-7B-Instruct-v0.2` performs on the above question. The standard prompt for Mistral-Instruct could be found [here](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2).

In [3]:
question = """What company advertised itself with the slogan "We'll leave a light on for you"?"""
template = "[INST] Answer the questions:\n\nQuestion: {question} [/INST] The answer is:"
prompt = template.format_map(dict(question=question))
print(prompt)

[INST] Answer the questions:

Question: What company advertised itself with the slogan "We'll leave a light on for you"? [/INST] The answer is:


In [4]:
input_ids = llm_tokenizer(prompt,return_tensors='pt').input_ids.to(device)
generated_output = llm.generate(
        input_ids = input_ids,
        do_sample=False,
        max_new_tokens=20,
        pad_token_id=llm_tokenizer.pad_token_id,
    )
result = llm_tokenizer.batch_decode(generated_output[:,input_ids.shape[1]:],skip_special_tokens=True)[0]
print(result)

Holiday Inn. Holiday Inn is a global hotel chain that has used the slogan "We


This is not a right answer!

## Latency
Let's calculate the latency with a larger batch number and batch size.

In [5]:
%%time
batch_size = 24
num_batch = 50
input_ids = input_ids.repeat(batch_size,1)
for _ in range(num_batch):
    generated_output = llm.generate(
            input_ids = input_ids,
            do_sample=False,
            max_new_tokens=20,
            pad_token_id=llm_tokenizer.pad_token_id,
        )

CPU times: user 30.1 s, sys: 24.4 ms, total: 30.1 s
Wall time: 30.1 s


## RAG

To get right answer, we need to retrieve relevant document for LLM. For illustration purpose, suppose our datastore have 5 documents, all from Wikipedia:

In [6]:
documents = [
    'Alvin and the Chipmunks | " Alvin and the Chipmunks, originally David Seville and the Chipmunks or simply The Chipmunks, are an American animated virtual band created by Ross Bagdasarian for a novelty record in 1958. The group consists of three singing animated anthropomorphic chipmunks named Alvin, Simon, and Theodore. They are managed by their human adoptive father, David ""Dave"" Seville. Bagdasarian provided the group\'s voices sped up to create high-pitched squeaky voices (which wasn\'t entirely new to him, having worked on ""Witch Doctor"" earned the record two Grammy Awards for engineering). ""The Chipmunk Song"" became a number-one single in the United States. After Bagdasarian died in 1972, the characters’ voices were provided by his son Ross Bagdasarian Jr. and the latter\'s wife Janice Karman in the subsequent incarnations of "',
    "Jamie Lee Curtis |  Jamie Lee Curtis (born November 22, 1958) is an American actress and writer. She is the recipient of several accolades, including a British Academy Film Award, two Golden Globe Awards and a star on the Hollywood Walk of Fame in 1998. Curtis made her film acting debut as Laurie Strode in John Carpenter's horror film Halloween (1978), which established her as a scream queen, and she thereafter appeared in a string of horror films, including The Fog, Prom Night, Terror Train (all 1980) and Roadgames (1981). She reprised the role of Laurie in the sequels Halloween II (1981), Halloween H20: 20 Years Later (1998), Halloween: Resurrection (2002), Halloween (2018), and Halloween Kills (2021). Her filmography is largely characterized by independent film that have been box-office successes, with 8 of her lead-actress credits ",
    'Sunset Boulevard (musical) | " The American premiere was at the Shubert Theatre in Century City, Los Angeles, California, on 9 December 1993, with Close as Norma and Alan Campbell as Joe. Featured were George Hearn as Max and Judy Kuhn as Betty. Lloyd Webber had reworked both the book and score, tightening the production, better organising the orchestrations, and adding the song ""Every Movie\'s a Circus"". This new production was better received by the critics and was an instant success, running for 369 performances. The Los Angeles production also recorded a new cast album that is well regarded. It is also the only unabridged cast recording of the show, since the original London recording was trimmed by over thirty minutes. A controversy arose with this production after Faye Dunaway was hired to replace Glenn Close. Dunaway went into rehearsals with Rex Smith as Joe and Jon Cypher as Max. Tickets "',
    'Arthur Balfour |  Balfour was appointed prime minister on 12 July 1902 while the King was recovering from his recent appendicitis operation. Changes to the Cabinet were thus not announced until 9 August, when the King was back in London. The new ministers were received in audience and took their oaths on 11 August.',
    'Motel 6 | " Beginning in 1986, Motel 6 has advertised through radio commercials featuring the voice of writer and National Public Radio commentator Tom Bodett, with the tagline "We\'ll leave the light on for you." The ads were created by Dallas advertising agency The Richards Group. They feature a tune composed by Tom Faulkner, performed by him on guitar and Milo Deering on fiddle. The first spots were conceived and written by David Fowler. In 1996, the ads won a Clio Award. The campaign itself has won numerous national and international awards and was selected by Advertising Age magazine as one of the Top 100 Advertising Campaigns of the Twentieth Century."',
]

## Setup Retriever
In modern dense retrieval system, a document is often encoded to a dense embedding with a document encoder, and this embedding is used for retrieval. In this part, we use `Salesforce/SFR-Embedding-Mistral`, the leading sentence emebdding model in [MTEB](https://huggingface.co/spaces/mteb/leaderboard).

In [7]:
retriever_name_or_path = "Salesforce/SFR-Embedding-Mistral"
retriever = SFR.from_pretrained(retriever_name_or_path,torch_dtype = torch.bfloat16).eval().to(device)
retriever_tokenizer = AutoTokenizer.from_pretrained(retriever_name_or_path)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
## get the embedding for each document
retriever_input = retriever_tokenizer(documents,max_length=180,padding=True,truncation=True,return_tensors='pt').to(device)
with torch.no_grad():
    doc_embeds = retriever.get_doc_embedding(input_ids=retriever_input.input_ids,attention_mask=retriever_input.attention_mask)
print(doc_embeds.shape)

torch.Size([5, 4096])


In [9]:
## now we have constructed a datastore with five docuements and their corresponding embeddings
datastore = (documents,doc_embeds)

In [10]:
## search over datastore
## 1. encode query
retriever_input = retriever_tokenizer(question,max_length=180,padding=True,truncation=True,return_tensors='pt').to(device)
with torch.no_grad():
    query_embed = retriever.get_query_embedding(input_ids=retriever_input.input_ids,attention_mask=retriever_input.attention_mask)
print(query_embed.shape)

torch.Size([1, 4096])


In [11]:
## 2. search over doc_embeds with dot product and take the top-1 document
_,index = torch.topk(torch.matmul(query_embed,doc_embeds.T),k=1)
top1_doc_index = index[0][0].item()
print(top1_doc_index)

4


In [12]:
## 3. fetch the document
relevant_doc = datastore[0][top1_doc_index]
print(relevant_doc)

Motel 6 | " Beginning in 1986, Motel 6 has advertised through radio commercials featuring the voice of writer and National Public Radio commentator Tom Bodett, with the tagline "We'll leave the light on for you." The ads were created by Dallas advertising agency The Richards Group. They feature a tune composed by Tom Faulkner, performed by him on guitar and Milo Deering on fiddle. The first spots were conceived and written by David Fowler. In 1996, the ads won a Clio Award. The campaign itself has won numerous national and international awards and was selected by Advertising Age magazine as one of the Top 100 Advertising Campaigns of the Twentieth Century."


In [13]:
## 4. concate the doc and query in a template
rag_template = """[INST] Refer to the background document and answer the questions:

Background: {document}

Question: {question} [/INST] The answer is:"""
prompt = rag_template.format_map(dict(document=relevant_doc,question=question))
print(prompt)

[INST] Refer to the background document and answer the questions:

Background: Motel 6 | " Beginning in 1986, Motel 6 has advertised through radio commercials featuring the voice of writer and National Public Radio commentator Tom Bodett, with the tagline "We'll leave the light on for you." The ads were created by Dallas advertising agency The Richards Group. They feature a tune composed by Tom Faulkner, performed by him on guitar and Milo Deering on fiddle. The first spots were conceived and written by David Fowler. In 1996, the ads won a Clio Award. The campaign itself has won numerous national and international awards and was selected by Advertising Age magazine as one of the Top 100 Advertising Campaigns of the Twentieth Century."

Question: What company advertised itself with the slogan "We'll leave a light on for you"? [/INST] The answer is:


In [14]:
## retrieval-augmented generation
input_ids = llm_tokenizer(prompt,return_tensors='pt').input_ids.to(device)
generated_output = llm.generate(
        input_ids = input_ids,
        do_sample=False,
        max_new_tokens=20,
        pad_token_id=llm_tokenizer.pad_token_id,
    )
result = llm_tokenizer.batch_decode(generated_output[:,input_ids.shape[1]:],skip_special_tokens=True)[0]
print(result)

Motel 6

Explanation: Motel 6 is the company that advertised


In [15]:
%%time
batch_size = 24
num_batch = 50
input_ids = input_ids.repeat(batch_size,1)
for _ in range(num_batch):
    generated_output = llm.generate(
            input_ids = input_ids,
            do_sample=False,
            max_new_tokens=20,
            pad_token_id=llm_tokenizer.pad_token_id,
        )

CPU times: user 42.7 s, sys: 2.22 s, total: 44.9 s
Wall time: 44.9 s


We got it! By retrieving the relevant document, LLM could now generate the right answer. However, we could also observe that propmt length is significantly extended. 

In [16]:
question_len = llm_tokenizer(question,return_length=True,add_special_tokens=False).length
doc_len = llm_tokenizer(relevant_doc,return_length=True,add_special_tokens=False).length
print(question_len,doc_len)

20 163


## xRAG
In xRAG, we could only use one soft token to replace the whole document. Specifically, we directly project document embedding into the LLM representation space.

In RAG, we have:
```
Embedding(doc+query), with length |doc|+|query|
```
In xRAG, we have:
```
Projector(doc_embedding)+Embedding(query), with length 1+|query|
```

In [17]:
## xrag
## after getting the top1_doc_index, we get the doc embedding
relevant_embedding = datastore[1][top1_doc_index]

## build prompt where XRAG_TOKEN is only a player holder taking up only one token
prompt = rag_template.format_map(dict(question=question,document=XRAG_TOKEN))
print(prompt)
input_ids = llm_tokenizer(prompt,return_tensors='pt').input_ids.to(device)
generated_output = llm.generate(
        input_ids = input_ids,
        do_sample=False,
        max_new_tokens=20,
        pad_token_id=llm_tokenizer.pad_token_id,
        retrieval_embeds = relevant_embedding.unsqueeze(0),
    )
result = llm_tokenizer.batch_decode(generated_output,skip_special_tokens=True)[0]
print(result)

[INST] Refer to the background document and answer the questions:

Background: <xRAG>

Question: What company advertised itself with the slogan "We'll leave a light on for you"? [/INST] The answer is:


Motel 6. The slogan was created in 1962 by Tom Bodett


In [18]:
%%time
batch_size = 24
num_batch = 50
input_ids = input_ids.repeat(batch_size,1)
retrieval_embeds = relevant_embedding.unsqueeze(0).repeat(batch_size,1)
for _ in range(num_batch):
    generated_output = llm.generate(
            input_ids = input_ids,
            do_sample=False,
            max_new_tokens=20,
            pad_token_id=llm_tokenizer.pad_token_id,
            retrieval_embeds = retrieval_embeds,
        )

CPU times: user 30.9 s, sys: 58.6 ms, total: 31 s
Wall time: 31 s


By only using one soft token, we could still the correct result! This is how xRAG works! xRAG also has the following advantages:
- do not need extra memory, since we reuse the document embedding---perviously only used for retrieval
- do not need extra computation, we simply use a two-layer MLP to project document emebdding
- do not need full-parameter tuning, we only train this projector